#### 1. Use BLINK to map the aspects to wikipedia (wiki_id)

##### An example : "Did not enjoy the new **Windows 8** touchscreen functions."

In [1]:
import blink.main_dense as main_dense
import argparse

In [2]:
models_path = "models/" # the path where you stored the BLINK models

config = {
    "test_entities": None,
    "test_mentions": None,
    "interactive": False,
    "top_k": 10,
    "biencoder_model": models_path+"biencoder_wiki_large.bin",
    "biencoder_config": models_path+"biencoder_wiki_large.json",
    "entity_catalogue": models_path+"entity.jsonl",
    "entity_encoding": models_path+"all_entities_large.t7",
    "crossencoder_model": models_path+"crossencoder_wiki_large.bin",
    "crossencoder_config": models_path+"crossencoder_wiki_large.json",
    "fast": True, # set this to be true if speed is a concern
    "output_path": "logs/" # logging directory
}

args = argparse.Namespace(**config)

models = main_dense.load_models(args, logger=None)

In [8]:
data_to_link = []
context_left = "Did not enjoy the new ".lower()
context_right = " touchscreen functions.".lower()
aspect = "Windows 8".lower()

process_data = {
    "id": 1,
    "label": "unknown",
    "label_id": -1,
    "context_left": context_left,
    "mention": aspect,
    "context_right":context_right,
}
data_to_link.append(process_data)    
_, _, _, _, _,predictions, scores, = main_dense.run(args, None, *models, test_data=data_to_link)

100%|██████████| 1/1 [00:01<00:00,  1.44s/it]


In [17]:
print('predictions : ', predictions)
print('scores : ', scores)

predictions :  [['Windows 8', 'Mac OS 8', 'Internet Explorer 8', 'Windows Phone 8', 'Windows 8.1', 'Windows 8 editions', 'IOS 8', 'Windows 8.x', 'Features new to Windows 8', 'IT8']]
scores :  [array([85.2276  , 81.668175, 81.086365, 81.07576 , 80.64415 , 80.434105,
       79.99263 , 79.93738 , 79.8428  , 79.51455 ], dtype=float32)]


In [16]:
entity_id = models[5][predictions[0][0]] # get the local_id from blink
wikipedia_id2local_id = models[8]
local_id2wikipedia_id = {value : key for key, value in wikipedia_id2local_id.items()} 
wiki_id = local_id2wikipedia_id[entity_id] # change local_id to wiki_id
print('entity_id:', entity_id)
print('wiki_id', wiki_id)

entity_id: 2912299
wiki_id 24806506


#### 2. After getting the wiki_id of aspects, obtain the corresponding neighbors from dbpedia

In [13]:
import requests
from SPARQLWrapper import SPARQLWrapper, JSON

def downloadByUrl(url, downloadPath, timeout=60):
    down_res = requests.get(url=url,timeout=(timeout-10,timeout))

    with open(downloadPath,"wb") as code:
        code.write(down_res.content)

def downloadByID(wikiId):
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    query = "SELECT ?thing { ?thing dbo:wikiPageID " + str(wikiId) + "}"
    try:
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        sparql.setTimeout(60)
        results = sparql.query().convert()
        print(results)
        dbpediaUrl = results["results"]["bindings"][0]["thing"]["value"]
        dbpediaCsvUrl = 'https://dbpedia.org/data/'+dbpediaUrl.split('/')[-1]+'.ntriples'

    except Exception as e:
        print(e)

    downloadPath = './' + f"{dbpediaUrl.split('/')[-1]}.csv"
    downloadByUrl(dbpediaCsvUrl, downloadPath)
    return downloadPath

##### Triplets of Windows_8 : 

In [14]:
path = downloadByID(wiki_id) # download 
with open(path, 'r', encoding='utf-8') as f:
    lines = f.readlines()
print(lines[10:15])

{'head': {'link': [], 'vars': ['thing']}, 'results': {'distinct': False, 'ordered': True, 'bindings': [{'thing': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Windows_8'}}]}}
['<http://dbpedia.org/resource/Windows_8>\t<http://dbpedia.org/property/wikiPageUsesTemplate>\t<http://dbpedia.org/resource/Template:Infobox_OS> .\n', '<http://dbpedia.org/resource/Windows_8>\t<http://www.w3.org/2002/07/owl#sameAs>\t<http://cs.dbpedia.org/resource/Windows_8> .\n', '<http://dbpedia.org/resource/Windows_8>\t<http://dbpedia.org/ontology/wikiPageWikiLink>\t<http://dbpedia.org/resource/DVD> .\n', '<http://dbpedia.org/resource/Windows_8>\t<http://dbpedia.org/ontology/wikiPageWikiLink>\t<http://dbpedia.org/resource/X64> .\n', '<http://dbpedia.org/resource/Windows_8>\t<http://www.w3.org/2000/01/rdf-schema#label>\t"Windows 8"@ru .\n']
